# Decision trees

* Decision trees algorithm can be used for both classification and regression problems, we talk about decision trees in classification problems first.

## What is it

* A decision tree consists of decision blocks and terminating blocks, decision blocks can lead to other decision blocks or terminating blocks through branchs. Each decision blocks represents a feature and the value of the feature decides which branch to go to. Terminating blocks represent labels.

#### Input
1. A labeled dataset ***A***.
2. An unlabeled datas sample ***x***.

#### Output
1. A decision tree ***T***.
2. Label for ***x***.

## How does it work

#### Definitions

1. Information gain: building a decision tree is all about how to split the dataset, the change in information before and after the split is called the information gain. We split the dataset the way that it gives the highest information gain.

2. Entropy (Shannon entropy): the way to calculation the information of a dataset.

    H = -sum(p(x_i)log2p(x_i))
    
    where i = 1...n, and p(x_i) is the probability of choosing this class

In [4]:
from math import log

def shannon_entropy(dataset: list) -> float:
    num_samples = len(dataset)
    label_counts = {}
    for sample in dataset:
        label = sample[-1]
        label_counts[label] = label_counts.get(label, 0) + 1
    entropy = 0.0
    for label, count in label_counts.items():
        prob = count / num_samples
        entropy -= prob * log(prob, 2)
    return entropy

There are two components in the above pseudo code: split the dataset and find the best feature to split

In [5]:
def split_dataset(dataset: list,
                  index: int,
                  value) -> list:
    '''
    :param list dataset: dataset to be splitted
    :param int index: feature index, which feature are we splitting on
    :param int/str/float value: feature value
    :return: list of rest dataset
    
    e.g.
    >>> myDat
    [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
    >>> trees.splitDataSet(myDat,0,1)
    [[1, 'yes'], [1, 'yes'], [0, 'no']]
    >>> trees.splitDataSet(myDat,0,0)
    [[1, 'no'], [1, 'no']]
    '''
    rest_dataset = []
    for sample in dataset:
        if sample[index] == value:
            # exclude this feature
            rest_feats = sample[:index]
            rest_feats.extend(sample[index+1:])
            # append this sample with reduced features
            rest_dataset.append(rest_feats)
    return rest_dataset

In [6]:
def best_feature_to_split(dataset: list):
    num_feats = len(dataset[0]) - 1
    base_entropy = shannon_entropy(dataset)
    best_info_gain = 0.0
    best_feat_index = -1
    for i in range(num_feats):
        feat_values = [sample[i] for sample in dataset]
        new_entropy = 0.0
        for value in set(feat_values):
            subset = split_dataset(dataset, i, value)
            prob = len(subset) / len(dataset)
            new_entropy += prob * shannon_entropy(subset)
        info_gain = base_entropy - new_entropy
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feat_index = i
    return best_feat_index

In [7]:
def majority_count(lables):
    lable_count = {}
    for lable in lables:
        lable_count[lable] = lable_count.get(lable, 0) + 1
    sorted_label_count = [k for k, v in sorted(lable_count.items(),
                                               key=lambda item: item[1],
                                               reverse=True)]
    return sorted_label_count[0]                         

In [8]:
def build_tree(dataset, features):
    labels = [sample[-1] for sample in dataset]
    if len(set(labels)) == 1:
        # only one label exists
        return labels[0]
    if len(dataset[0]) == 1:
        # no more features
        return majority_count(dataset)
    best_feat_index = best_feature_to_split(dataset)
    best_feat = features[best_feat_index]
    # new tree node
    tree = {best_feat: {}}
    features.remove(best_feat)
    feat_values = [sample[best_feat_index] for sample in dataset]
    for value in set(feat_values):
        subfeatures = features[:]
#         import pdb; pdb.set_trace()
        tree[best_feat][value] = build_tree(split_dataset(dataset, best_feat_index, value), subfeatures)
    return tree

## Simple example

In [18]:
def create_dataset():
    dataset = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    features = ['no surfacing', 'flippers']
    return dataset, features

In [19]:
dataset, features = create_dataset()
tree = build_tree(dataset, features)

In [20]:
tree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [21]:
def classify_decision_tree(tree, features, test_data):
    root_feature = list(tree.keys())[0]
    child_tree = tree[root_feature]
    root_feature_index = features.index(root_feature)
    for key in child_tree.keys():
        if test_data[root_feature_index] == key:
            if isinstance(child_tree[key], dict):
                # not leaf yet
                return classify_decision_tree(child_tree[key], features, test_data)
            else:
                return child_tree[key]

In [23]:
features = ['no surfacing', 'flippers']
classify_decision_tree(tree, features, [1, 0])

'no'